In [1]:
import tensorflow as tf
import numpy as np

: 

In [ ]:
import csv
time_step =[]
temps = []
temporanea=[]
with open('data/heart.csv') as file:
  reader = csv.reader(file, delimiter=',')
  next(reader)
  i=0
 
  for row in reader:
    
    temps.append([int(row[0]),int(row[1]),int(row[2]),int(row[3]),int(row[4]),int(row[5]),int(row[6]),int(row[7]),int(row[8]),float(row[9]),int(row[10]),int(row[11]),int(row[12])])
    time_step.append(int(row[13]))

series = np.array(temps)
time = np.array(time_step)

print(len(time))


303


In [ ]:
# Split the dataset into a training, validation and test partition.
split_time = 44
time_train = time[split_time:]
x_train = series[split_time:]
time_valid = time[:split_time-20]
x_valid = series[:split_time-20]
x_test=series[24:split_time]
time_test=time[24:split_time]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, time_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, time_valid))
test_dataset =tf.data.Dataset.from_tensor_slices((x_test, time_test))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.8, min_lr=0.0000001)

In [ ]:
# Define a model
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, activation="relu",kernel_regularizer=tf.keras.regularizers.L2(),bias_regularizer=tf.keras.regularizers.L2()),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.L2(),bias_regularizer=tf.keras.regularizers.L2()),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.05),

  tf.keras.layers.Dense(8, activation="relu",kernel_regularizer=tf.keras.regularizers.L2(),bias_regularizer=tf.keras.regularizers.L2()),
  
  tf.keras.layers.Dense(1,activation="sigmoid"),
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer , loss = 'binary_crossentropy' , metrics = ['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=100,validation_data=valid_dataset,callbacks=[learning_rate_reduction])
while (model.evaluate(test_dataset)[1]<0.65):                                 
    model.fit(train_dataset, epochs=1,validation_data=valid_dataset,callbacks=[learning_rate_reduction])


Epoch 1/100


2023-05-07 04:01:48.162862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [259,13]
	 [[{{node Placeholder/_0}}]]
2023-05-07 04:01:48.165386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [259]
	 [[{{node Placeholder/_1}}]]


5/5 [==============================] - ETA: 0s - loss: 4.3586 - accuracy: 0.4440

2023-05-07 04:01:59.020442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [24]
	 [[{{node Placeholder/_1}}]]


5/5 [==============================] - 13s 681ms/step - loss: 4.3586 - accuracy: 0.4440 - val_loss: 3.4427 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/100
5/5 [==============================] - 0s 64ms/step - loss: 3.8151 - accuracy: 0.6023 - val_loss: 3.3510 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/100
5/5 [==============================] - 0s 70ms/step - loss: 3.3660 - accuracy: 0.7606 - val_loss: 3.1992 - val_accuracy: 0.0417 - lr: 0.0010
Epoch 4/100
3/5 [=================>............] - ETA: 0s - loss: 3.0048 - accuracy: 0.7292
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
5/5 [==============================] - 0s 46ms/step - loss: 2.9637 - accuracy: 0.7297 - val_loss: 2.8853 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/100
5/5 [==============================] - 0s 47ms/step - loss: 2.6159 - accuracy: 0.7297 - val_loss: 2.7481 - val_accuracy: 0.0000e+00 - lr: 8.0000e-04
Epoch 6/100
5/5 [==============================] - 0s 43ms/step - loss: 2.351

2023-05-07 04:02:51.319908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [20]
	 [[{{node Placeholder/_1}}]]


In [ ]:
# Evaluate the model
model.evaluate(test_dataset)

1/1 [==============================] - 0s 56ms/step - loss: 0.9478 - accuracy: 0.7500


[0.9478428959846497, 0.75]